In [1]:
# !pip3 install open3d --break-system-packages
# !pip3 install plyfile --break-system-packages
# !pip3 install pyntcloud --break-system-packages

In [2]:
# !pip3 list

In [3]:
# !pip3 uninstall -y meshlib --break-system-packages
# !pip3 uninstall -y SQLAlchemy --break-system-packages
# !pip3 uninstall -y sqlglot sqlglotrs --break-system-packages
# !pip3 uninstall -y rfc3986-validator rfc3339-validator --break-system-packages
# !pip3 uninstall -y PyWavelets --break-system-packages
# !pip3 uninstall -y numba --break-system-packages
# !pip3 uninstall -y numpy pandas


In [4]:
# !pip3 install numpy==1.26.4 pandas==2.2.2 --break-system-packages

# Import Library

In [5]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import sys
import ast
import copy

import numpy as np
np.set_printoptions(suppress=True)

import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from plyfile import PlyData, PlyElement
from collections import Counter

from scipy.spatial import ConvexHull
from scipy.spatial import cKDTree
from pyntcloud import PyntCloud

from Utilities.utilities import read_points3D, write_ply, calculate_volume, calculate_volume_file

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Function Plot

In [6]:
def plot_mesh(mesh):
    vertices = np.asarray(mesh.vertices)
    
    # Plot the PLY file
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=0.005)
    
    # Set axis labels
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    # Set title
    ax.set_title('3D Mesh Plot')

    plt.show()
    return None

# User Defined Function

In [7]:
#################################################
# Remove isolated mesh
#################################################
def remove_isolated_vertices(mesh, eps=1.0, min_points=10):
    print("\nRemoving isolated vertices using DBSCAN clustering")

    # Convert mesh to point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = mesh.vertices

    # Apply DBSCAN clustering
    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))

    # Keep only the largest cluster
    largest_cluster = np.argmax(np.bincount(labels[labels >= 0]))
    mask = labels == largest_cluster

    # Map old indices to new indices
    index_map = -np.ones(len(mask), dtype=int)
    index_map[mask] = np.arange(np.sum(mask))

    # Filter vertices
    filtered_vertices = np.asarray(mesh.vertices)[mask]

    # Filter triangles
    triangles = np.asarray(mesh.triangles)
    filtered_triangles = []
    for tri in triangles:
        if np.all(mask[tri]):
            filtered_triangles.append(index_map[tri])

    # Create cleaned mesh
    cleaned_mesh = o3d.geometry.TriangleMesh()
    cleaned_mesh.vertices = o3d.utility.Vector3dVector(filtered_vertices)
    cleaned_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)

    print(f"Remaining vertices: {len(filtered_vertices)}")
    return cleaned_mesh


In [8]:
#################################################
# Shift to positive
#################################################
def shift_to_positive(mesh):
    vertices = np.asarray(mesh.vertices)
    min_coords = vertices.min(axis=0)
    shifted_vertices = vertices - min_coords # shift so that min_coords becomes (0, 0, 0)
    
    # shift = np.abs(np.minimum(min_coords, 0))
    # shifted_vertices = vertices + shift

    shifted_mesh = o3d.geometry.TriangleMesh()
    shifted_mesh.vertices = o3d.utility.Vector3dVector(shifted_vertices)
    shifted_mesh.triangles = mesh.triangles

    return shifted_mesh


In [9]:
#################################################
# Reorientate Mesh (kinda works)
#################################################
def find_floor_plane_and_reorient(mesh, iteration=10):
    print(f'\nLocating floor plane and reorientate to x-y plane')

    for i in range(iteration):
        # Convert mesh vertices to numpy array
        points = np.asarray(mesh.vertices)

        # Create a PointCloud object from the mesh vertices
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(points)

        # Use RANSAC to find the plane in the point cloud
        plane_model, inliers = point_cloud.segment_plane(
            distance_threshold=0.01,
            ransac_n=3,
            num_iterations=1000)
        
        # Extract the plane coefficients
        a, b, c, d = plane_model
        
        # Calculate the normal vector of the plane
        normal_vector = np.array([a, b, c])
        
        # Calculate the rotation matrix to align the normal vector with the z-axis
        z_axis = np.array([0, 0, 1])
        rotation_axis = np.cross(normal_vector, z_axis)
        rotation_angle = np.arccos(np.dot(normal_vector, z_axis))
        rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(
            rotation_axis * rotation_angle)
        
        # Rotate the mesh
        mesh.rotate(rotation_matrix, center=(0, 0, 0))
        
        # Translate the mesh to ensure the center of the floor is at (0, 0, 0)
        points = np.asarray(mesh.vertices)

        floor_center = points[inliers].mean(axis=0)
        translation_vector = -floor_center
        mesh.translate(translation_vector)
        
        # Ensure all points are on the positive side of z-axis by rotating about the x-y plane
        points = np.asarray(mesh.vertices)

        min_z = points[:, 2].min()
        if min_z < 0:
            rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz([np.pi, 0, 0])
            mesh.rotate(rotation_matrix, center=(0, 0, 0))
        
        # To handle mesh and point cloud
        mean_z = np.mean(np.asarray(mesh.vertices)[:, 2]) 
        if mean_z < 0:
            rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz([np.pi, 0, 0])
            mesh.rotate(rotation_matrix, center=(0, 0, 0))
            
        # print(f'Rotation {i+1}')
        # print(np.round(rotation_matrix), 5)
        # print(rotation_matrix)
        
    print(f'No of points: {len(np.asarray(mesh.vertices))}')
        
    return mesh

In [10]:
#################################################
# Remove floor
#################################################
def crop_mesh_z_axis(mesh, percentile=10):
    print('\nCropping mesh below the bottom 10th percentile of Z values')

    # Convert mesh vertices to numpy array
    points = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    # Calculate Z range and threshold
    z_values = points[:, 2]
    z_min, z_max = np.min(z_values), np.max(z_values)
    z_range = z_max - z_min
    z_threshold = z_min + percentile / 100 * z_range # bottom 10% threshold

    # Create a mask to keep points with Z >= threshold
    mask = z_values >= z_threshold

    # Map old indices to new indices
    index_map = -np.ones(len(points), dtype=int)
    index_map[mask] = np.arange(np.sum(mask))

    # Filter points
    filtered_points = points[mask]

    # Filter triangles: keep only those where all vertices are retained
    filtered_triangles = []
    for triangle in triangles:
        if np.all(mask[triangle]):
            new_triangle = index_map[triangle]
            filtered_triangles.append(new_triangle)

    # Create a new mesh
    cropped_mesh = o3d.geometry.TriangleMesh()
    cropped_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
    cropped_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)

    print(f'No of points: {len(filtered_points)}')
    return cropped_mesh


In [11]:

#################################################
# Remove smallest objects
#################################################
def remove_small_objects(mesh):
    print(f'\nPerform mesh clustering to remove small objects')
    
    # Convert mesh vertices to numpy array
    points = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    
    # Create a PointCloud object from the mesh vertices
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    
    print('Range:', point_cloud.get_max_bound() - point_cloud.get_min_bound())
    
    # Cluster the points using DBSCAN algorithm
    eps = 0.1 # 0.01 (all noise)
    min_points = 10
    print('eps:', eps)
    print('min_points', min_points)    
    
    labels = np.array(point_cloud.cluster_dbscan(eps=eps,
                                                    min_points=min_points,
                                                    print_progress=True))
    
    print('Counter:')
    print(Counter(labels))
    
    # Check the number of clusters
    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    
    # If only one cluster, return the one
    if num_clusters <= 1:
        print('Only 1 cluster found!')
        print(f'No of points: {len(np.asarray(mesh.vertices))}')
        return mesh
    
    print(f'{num_clusters} cluster found!')
    
    # Find the largest cluster
    largest_cluster_label = np.argmax(np.bincount(labels[labels >= 0]))
    
    # Create a mask to filter out points that are not in the largest cluster
    mask = labels == largest_cluster_label
    
    # Apply the mask to get the filtered points
    filtered_points = points[mask]
    
    # Remap the indices of the triangles to the filtered points
    index_map = np.zeros(len(points), dtype=int) - 1
    index_map[mask] = np.arange(np.sum(mask))
    filtered_triangles = []
    for triangle in triangles:
        if np.all(triangle < len(mask)) and np.all(mask[triangle]):
            filtered_triangles.append(index_map[triangle])
    
    # Create a new mesh with the filtered points and triangles
    largest_object_mesh = o3d.geometry.TriangleMesh()
    largest_object_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
    largest_object_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)
    
    # If the amount of mesh removed is over 10% of the original mesh, stop the removal process    
    if len(np.asarray(largest_object_mesh.vertices)) * 10 < len(np.asarray(mesh.vertices)):
        print(f'Too many points are removed! Undo step')
        print(f'New - No of points: {len(np.asarray(largest_object_mesh.vertices))}')
        print(f'Old - No of points: {len(np.asarray(mesh.vertices))}')
        return mesh
    
    print(f'No of points: {len(np.asarray(largest_object_mesh.vertices))}')
    
    return largest_object_mesh

In [12]:
#################################################
# Rescaling based on foot
#################################################
def get_distance(x1, y1, x2, y2):
    distance = ((x1 - x2)**2 + (y1 - y2)**2)**0.5
    return distance
    

def scale_longest_xy_axis(mesh, target_length, percentile=100):
    print(f'\nScaling longest x-y axis based on foot length: {target_length}cm')

    points = np.asarray(mesh.vertices)
    z_threshold = np.percentile(points[:, 2], percentile)
    bottom_points = points[points[:, 2] <= z_threshold]

    # Get bounding box in x-y plane
    x_min, y_min = np.min(bottom_points[:, :2], axis=0)
    x_max, y_max = np.max(bottom_points[:, :2], axis=0)

    p_min = np.array([x_min, y_min, 0])
    p_max = np.array([x_max, y_max, 0])

    # Estimate longest distance as diagonal of bounding box
    p1x = (x_max + x_min) / 2
    p1y = y_min
    
    p2x = (x_max + x_min) / 2
    p2y = y_max
    
    p3x = x_min
    p3y = (y_max + y_min) / 2
    
    p4x = x_max
    p4y = (y_max + y_min) / 2
    
    l1 = get_distance(p1x, p1y, p2x, p2y)
    l2 = get_distance(p3x, p3y, p4x, p4y)

    # Get max distance
    if l1 > l2:    
        longest_distance = l1
    else:
        longest_distance = l2

    scale_factor = target_length / longest_distance #* 1.2 #* np.sqrt(2)
    print(f'Scaling Factor: {round(scale_factor, 1)} - Estimated Distance: {round(longest_distance, 1)} - Target Length: {target_length}')

    # Get points
    if l1 > l2:    
        point1 = np.array([p1x * scale_factor, p1y * scale_factor, 0])
        point2 = np.array([p2x * scale_factor, p2y * scale_factor, 0])
    else:
        point1 = np.array([p3x * scale_factor, p3y * scale_factor, 0])
        point2 = np.array([p4x * scale_factor, p4y * scale_factor, 0])

    p_min = np.array([x_min * scale_factor, y_min * scale_factor, 0])
    p_max = np.array([x_max * scale_factor, y_max * scale_factor, 0])

    points *= scale_factor
    mesh.vertices = o3d.utility.Vector3dVector(points)

    print(f'No of points: {len(points)}')
    return mesh, point1, point2, p_min, p_max

In [13]:
#################################################
# Reorientate foot (Take previous 2 points to orientate)
#################################################
def reorient_foot(mesh, point1, point2, target_direction=np.array([0, 1])):
    print(f'\nReorientate foot based on foot direction')
    
    points = np.asarray(mesh.vertices)

    # Compute the direction vector from point1 to point2 in the x-y plane
    direction_vector = point2[:2] - point1[:2]
    direction_vector /= np.linalg.norm(direction_vector)  # Normalize

    # Compute angle to target direction
    angle = np.arccos(np.dot(direction_vector, target_direction))

    # Determine rotation direction using cross product
    if np.cross(direction_vector, target_direction) < 0:
        angle = -angle

    # Rotation matrix around z-axis
    R = np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0,              0,             1]
    ])

    # Apply rotation
    rotated_points = points @ R.T
    mesh.vertices = o3d.utility.Vector3dVector(rotated_points)
    
    print(f'No of points: {len(np.asarray(mesh.vertices))}')
    
    return mesh

In [14]:
# #################################################
# # Crop Ankle
# #################################################
# def crop_mesh_z_axis_ankle(mesh, z_threshold = 10):
#     print(f'\nCropping mesh from the top to make it {z_threshold}cm')

#     # Convert mesh vertices to numpy array
#     points = np.asarray(mesh.vertices)
#     triangles = np.asarray(mesh.triangles)
    
#     max_value = np.max(points[:, 2])
#     print('Max value:', max_value)
    
#     if max_value < z_threshold:
#         print(f'No of points: {len(np.asarray(mesh.vertices))}')
#         return mesh
    
#     else:
#         # Create a mask to filter out points below the z_threshold
#         mask = points[:, 2] < z_threshold
        
#         # Apply the mask to get the filtered points and update the triangles accordingly
#         filtered_points = points[mask]
#         filtered_triangles = []
#         for triangle in triangles:
#             if np.all(mask[triangle]):
#                 filtered_triangles.append(triangle)
        
#         # Create a new mesh with the filtered points and triangles
#         cropped_mesh = o3d.geometry.TriangleMesh()
#         cropped_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
#         cropped_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)
        
#         print(f'No of points: {len(np.asarray(cropped_mesh.vertices))}')
#         return cropped_mesh

In [15]:
#################################################
#  Crop Ankle
#################################################
def crop_mesh_z_axis_ankle(mesh, z_threshold=10):
    print(f'\nCropping mesh from the top')

    # Convert mesh vertices to numpy array
    points = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    max_value = np.max(points[:, 2])
    print('Max value:', max_value)
    
    if max_value < z_threshold:
        print(f'No of points: {len(np.asarray(mesh.vertices))}')
        return mesh

    # Create a mask to keep points with Z >= threshold
    mask = points[:, 2] < z_threshold

    # Map old indices to new indices
    index_map = -np.ones(len(points), dtype=int)
    index_map[mask] = np.arange(np.sum(mask))

    # Filter points
    filtered_points = points[mask]

    # Filter triangles: keep only those where all vertices are retained
    filtered_triangles = []
    for triangle in triangles:
        if np.all(mask[triangle]):
            new_triangle = index_map[triangle]
            filtered_triangles.append(new_triangle)

    # Create a new mesh
    cropped_mesh = o3d.geometry.TriangleMesh()
    cropped_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
    cropped_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)

    print(f'No of points: {len(filtered_points)}')
    return cropped_mesh


In [16]:
#################################################
# Solid mesh
#################################################
def solidify_mesh_with_fans(mesh, z_min=0.0, z_max=10.0):
    print(f'\nSolidify mesh')
    points = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    # Keep only points within the z bounds
    mask = (points[:, 2] >= z_min) & (points[:, 2] <= z_max)
    valid_indices = np.where(mask)[0]
    index_map = {old_idx: new_idx for new_idx, old_idx in enumerate(valid_indices)}
    filtered_points = points[mask]

    # Remap triangle indices
    filtered_triangles = []
    for tri in triangles:
        if all(idx in index_map for idx in tri):
            new_tri = [index_map[idx] for idx in tri]
            filtered_triangles.append(new_tri)

    cap_triangles = []

    # Cap both ends using triangle fans
    for z_target in [z_min, z_max]:
        cap_indices = np.where(np.isclose(filtered_points[:, 2], z_target))[0]
        if len(cap_indices) >= 3:
            cap_ring = filtered_points[cap_indices]
            center = np.mean(cap_ring, axis=0)
            center_idx = len(filtered_points)
            filtered_points = np.vstack([filtered_points, center])

            # Sort cap points in counter-clockwise order
            xy = cap_ring[:, :2] - center[:2]
            angles = np.arctan2(xy[:, 1], xy[:, 0])
            sorted_indices = cap_indices[np.argsort(angles)]

            for i in range(len(sorted_indices)):
                i1 = sorted_indices[i]
                i2 = sorted_indices[(i + 1) % len(sorted_indices)]
                if z_target == z_min:
                    cap_triangles.append([i1, i2, center_idx])
                else:
                    cap_triangles.append([i2, i1, center_idx])  # flip normal

    all_triangles = np.vstack([filtered_triangles, cap_triangles])

    solid_mesh = o3d.geometry.TriangleMesh()
    solid_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
    solid_mesh.triangles = o3d.utility.Vector3iVector(all_triangles)
    solid_mesh.compute_vertex_normals()
    
    print(f'No of points: {len(np.asarray(solid_mesh.vertices))}')
    
    return solid_mesh



In [17]:
def plot_all_meshes(
    subject, data_image, foot_length, file_name, volume, 
    point1, point2,
    meshes, list_title):
    
    if file_name == 'File_Path_SFM':
        s = 1
    else:
        s = 0.008
    
    num_meshes = len(meshes)
    # fig = plt.figure(figsize=(15, 5 * num_meshes))
    fig = plt.figure(figsize=(16, 10))
    
    for i, mesh in enumerate(meshes):
        # ax = fig.add_subplot(1, num_meshes, i + 1, projection='3d')
        ax = fig.add_subplot(2, 3, i + 1, projection='3d')
        vertices = np.asarray(mesh.vertices)
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=s)
        
        # if list_title[i] in ['Rescaled Mesh']: # 'Remove Small Objects'
        #     ax.scatter(
        #         [point1[0], point2[0]], 
        #         [point1[1], point2[1]], 
        #         [point1[2], point2[2]], 
        #         s=s*10, color='red', label='Longest Distance Points')
            
        #     ax.plot(
        #         [point1[0], point2[0]], 
        #         [point1[1], point2[1]], 
        #         [point1[2], point2[2]], 
        #         color='green', label='Longest Distance Line')
        
        # Set axis labels
        ax.set_xlabel('X axis')
        ax.set_ylabel('Y axis')
        ax.set_zlabel('Z axis')
        
        # Set title
        # ax.set_title(f'3D Mesh Plot {i + 1}')
        # ax.set_title(f'({i+1}) {list_title[i]}\nPoints: {len(vertices)}')
        ax.set_title(f'({i+1}) {list_title[i]}', fontsize=18)
    
    
    # plt.suptitle(f'Subject {subject} - {data_image}\nFoot Length: {foot_length}cm - Points: {len(vertices)}\nVolume: {round(volume, 1)}cm3')
    # plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {foot_length}cm\nVolume: {round(volume, 1)}cm3')
    # plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {foot_length}cm')
    plt.tight_layout()
    # fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.3)

    folder_save_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Details/Feet_Preprocessing/{file_name}'
    if not os.path.exists(folder_save_path):
        os.makedirs(folder_save_path)

    save_path = f'{folder_save_path}/subject_{subject}_{data_image}_P.png'
    fig.savefig(save_path, dpi=300)  # Save at high resolution
    plt.close()
    # plt.show()

In [18]:
def plot_mesh_angles(subject, data_image, foot_length, file_name, 
                     volume,
                     point1, point2, 
                     p_min, p_max,
                     mesh, directions):
    
    if file_name == 'File_Path_SFM':
        s = 1
    else:
        s = 0.008
    
    num_directions = len(directions)

    # fig = plt.figure(figsize=(15, 5 * num_meshes))

    if num_directions == 6:
        fig = plt.figure(figsize=(16, 10))
    else:
        fig = plt.figure(figsize=(10, 10))
    
    for i, direction  in enumerate(directions):
        if num_directions == 6:
            ax = fig.add_subplot(2, 3, i + 1, projection='3d')
        else:
            ax = fig.add_subplot(2, 2, i + 1, projection='3d')
            
        vertices = np.asarray(mesh.vertices)
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=s)

        # # Plot foot length line        
        # ax.scatter(
        #     [point1[0], point2[0]], 
        #     [point1[1], point2[1]], 
        #     [point1[2], point2[2]], 
        #     s=s*100, color='green', label='Longest Distance Points')
        
        # ax.plot(
        #     [point1[0], point2[0]], 
        #     [point1[1], point2[1]], 
        #     [point1[2], point2[2]], 
        #     color='green', label='Longest Distance Line')
        
        # ax.scatter(
        #     [p_min[0], p_max[0]], 
        #     [p_min[1], p_max[1]], 
        #     [p_min[2], p_max[2]], 
        #     s=s*100, color='red', label='Longest Distance Points')
        
        # Set axis labels
        ax.set_xlabel('X axis')
        ax.set_ylabel('Y axis')
        ax.set_zlabel('Z axis')
        
        # Set title
        # ax.set_title(f'3D Mesh Plot {i + 1}')
    
        # Interpret direction as [elev, azim]
        if direction is not None:
            elev, azim = direction
            ax.view_init(elev=elev, azim=azim)
            ax.set_title(f'elev:{elev}, azim:{azim}', fontsize=18)
        else:
            ax.set_title('Original View', fontsize=18)

    # plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {foot_length}cm - Points: {len(vertices)}\nVolume: {round(volume, 1)}cm3')
    # plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {foot_length}cm - Points: {len(vertices)}')
    # plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {foot_length}cm')
    # plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {foot_length}cm\nVolume: {round(volume, 1)}cm3')
    plt.tight_layout()
    # fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.3)

    folder_save_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Details/Feet_Orientation/{file_name}'
    if not os.path.exists(folder_save_path):
        os.makedirs(folder_save_path)

    save_path = f'{folder_save_path}/subject_{subject}_{data_image}_O.png'
    fig.savefig(save_path, dpi=300)  # Save at high resolution
    plt.close()

    # plt.show()

# Main

In [19]:
df = pd.read_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_2_perfect_subjects.csv')
# df = pd.read_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_2_all_subjects.csv')
df_subject = pd.read_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Details/data_0_summary_grouped.csv')

df

,Subject,Image Data,Type,File_Path,Volume
0,1,1D0L,SFM,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,49.662720
1,1,1D0L,MVS_mesh,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,71.397707
2,1,1D0L,MVS_dense,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,71.156429
3,1,1D0L,MVS_refine,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,71.157388
4,1,1D0R,SFM,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,6.053574
...,...,...,...,...,...
223,46,46D0R,MVS_refine,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,2.141923
224,74,74D0R,SFM,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,2268.287191
225,74,74D0R,MVS_mesh,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,155.341813
226,74,74D0R,MVS_dense,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,154.125191


In [20]:
# # Testing
# data_type = 'SFM'
# index = 10

# df_temp = df[df['Type'] == data_type].reset_index(drop=True)
# subject = df_temp['Subject'][index]
# data_image = df_temp['Image Data'][index]
# ply_path = df_temp['File_Path'][index]

# file_name = ply_path.split('/')[-1]

# if 'L' in data_image:
#     column = 'Left foot length (cm)'
# else:
#     column = 'Right foot length (cm)'

# list_foot_length = df_subject[df_subject['Subject ID'] == subject][column]
# list_foot_length = list_feet_length.tolist()[0]
# list_foot_length = list_feet_length.replace(', nan', '').replace('nan', '')
# list_feet_length = ast.literal_eval(list_feet_length)
# feet_length = list_feet_length[0] # First

# print('#'*60)
# print(f'Subject {subject} - {data_image} - Feet length {feet_length}cm')
# print(f'{file_name}')
# print(f'{list_feet_length} - {feet_length}')

# #################################################
# # Load PLY file using Open3D
# #################################################
# if data_type != 'File_Path_SFM':
#     mesh = o3d.io.read_triangle_mesh(ply_path) # Read mesh
# else:
#     mesh = o3d.io.read_point_cloud(ply_path) # Read point dense cloud
#     print('Number of Vertices: ', len(mesh.points))

In [ ]:
list_data_type = [
    # 'SFM', 
    # 'MVS_mesh', 
    # 'MVS_dense', 
    'MVS_refine'
    ]

df_results = []

#################################################
# Run data type
#################################################
for data_type in list_data_type:
    
    #################################################
    # Run subjects
    #################################################
    df_temp = df[df['Type'] == data_type].reset_index(drop=True)
    
    for index, row in df_temp.iterrows():
        subject = df_temp['Subject'][index]
        data_image = df_temp['Image Data'][index]
        ply_path = df_temp['File_Path'][index]
        file_name = ply_path.split('/')[-1]

        if 'L' in data_image:
            column = 'Left'
        else:
            column = 'Right'
        
        try:
            list_feet_length = df_subject[df_subject['Subject ID'] == subject][f'{column} foot length (cm)']
            list_feet_length = list_feet_length.tolist()[0]
            list_feet_length = list_feet_length.replace(', nan', '').replace('nan', '')
            list_feet_length = ast.literal_eval(list_feet_length)
            feet_length = list_feet_length[0] # First
            # feet_length = np.mean(feet_length) # Mean
        except:
            feet_length = 25
        
        print('')
        print('#'*60)
        print(f'Subject {subject} - {data_image} - Feet length {feet_length}cm')
        print('#'*60)
        print(f'{file_name}')
        print(f'{list_feet_length} - {feet_length}')
        
        try:
            #################################################
            # Load PLY file using Open3D
            #################################################
            if data_type != 'File_Path_SFM':
                print('\nLoad Mesh')
                mesh = o3d.io.read_triangle_mesh(ply_path) # Read mesh
                print('Number of Vertices: ', len(mesh.vertices))
                
                # Downsample to a target number of triangles
                # print('\nDown sample mesh')
                # target_triangle_count = 50000 # adjust as needed
                # mesh = mesh.simplify_quadric_decimation(target_triangle_count)
                # print('Number of Vertices: ', len(mesh.vertices))

                
            else:
                print('\nLoad Point Dense Cloud')
                mesh = o3d.io.read_point_cloud(ply_path) # Read point dense cloud
                print('Number of Points: ', len(mesh.points))

                # If more than 2000 points, randomly sample
                points = np.asarray(mesh.points)
                if len(points) > 3000:
                    indices = np.random.choice(len(points), size=2000, replace=False)
                    mesh = mesh.select_by_index(indices)

                # mesh = mesh.voxel_down_sample(voxel_size=0.01)
                print('Number of Points (Down Sample): ', len(mesh.points))

                mesh.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)) # Estimate normals
                mesh.orient_normals_consistent_tangent_plane(100) # Optionally, orient the normals consistently
                
                distances = mesh.compute_nearest_neighbor_distance()
                avg_dist = np.mean(distances)
                radii = [avg_dist, avg_dist * 2]

                radii = [0.05, 0.1, 0.2] # Define radii for the Ball Pivoting algorithm
                mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
                    mesh, 
                    o3d.utility.DoubleVector(radii)
                    ) # Apply the Ball Pivoting algorithm

            #################################################
            # Pipeline to edit mesh
            #################################################
            mesh = remove_isolated_vertices(mesh)
            mesh = shift_to_positive(mesh)
            
            reoriented_mesh = find_floor_plane_and_reorient(copy.deepcopy(mesh))
            reoriented_mesh = shift_to_positive(reoriented_mesh)
            
            cropped_mesh = crop_mesh_z_axis(copy.deepcopy(reoriented_mesh), percentile=17) # 21
            cropped_mesh = shift_to_positive(cropped_mesh)
            
            largest_object_mesh = remove_small_objects(copy.deepcopy(cropped_mesh))
            largest_object_mesh = shift_to_positive(largest_object_mesh)
            
            scaled_mesh, point1, point2, p_min, p_max = scale_longest_xy_axis(copy.deepcopy(largest_object_mesh), feet_length)
            scaled_mesh = shift_to_positive(scaled_mesh)
            
            # rotate_feet_mesh = reorient_feet(copy.deepcopy(scaled_mesh), point1, point2)
            
            cropped_ankle_mesh = crop_mesh_z_axis_ankle(copy.deepcopy(scaled_mesh), z_threshold=12)
            # cropped_ankle_mesh = crop_mesh_z_axis_ankle(scaled_mesh, z_threshold=12)
            # cropped_ankle_mesh = copy.deepcopy(scaled_mesh)
            cropped_ankle_mesh = shift_to_positive(cropped_ankle_mesh)
            
            # solid_mesh = solidify_mesh_with_fans(copy.deepcopy(cropped_ankle_mesh))


            #################################################
            # Clean output
            #################################################
            final_mesh = copy.deepcopy(cropped_ankle_mesh)
            final_mesh = shift_to_positive(final_mesh)
            print('\nCleaning output')
            final_mesh.remove_duplicated_vertices()
            final_mesh.remove_degenerate_triangles()
            final_mesh.remove_unreferenced_vertices()
            final_mesh.remove_non_manifold_edges()
            final_mesh_vertices = np.asarray(final_mesh.vertices)
            
            # Check if the mesh is still valid
            if not mesh.is_edge_manifold():
                print("Warning: Mesh is not edge-manifold.")
            if not mesh.is_vertex_manifold():
                print("Warning: Mesh is not vertex-manifold.")

            print(f'No of points: {len(np.asarray(final_mesh.vertices))}')


            #################################################
            # Saving results
            #################################################
            print('\nSaving Final Mesh')
            path_save = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_edit/{subject}/{data_image}'
            if not os.path.exists(path_save):
                os.makedirs(path_save)
            o3d.io.write_triangle_mesh(f'{path_save}/postprocessed_{file_name}', final_mesh)

            # Visualize the PLY file
            # o3d.visualization.draw_geometries([reoriented_mesh])
            # plot_mesh(reoriented_mesh)

            #################################################
            # Saving results
            #################################################
            print(f'\nCalculating Volume')
            try:
                volume = calculate_volume_file(
                    file_path = f'{path_save}', 
                    file_name = f'postprocessed_{file_name}'
                    )
                error = ''
            except Exception as error:
                print(error)
                volume = 0
            
            print(f'{file_name} - Volume: {round(volume, 3)} cm3')

            #################################################
            # Plot processed meshes
            #################################################            
            plot_all_meshes(
                subject, data_image, feet_length, file_name.replace('.ply', ''), volume,
                point1, point2,
                [
                    mesh, reoriented_mesh, cropped_mesh, 
                    largest_object_mesh, scaled_mesh, cropped_ankle_mesh
                ],
                [
                    'Initial', 'Reorientate Mesh to Align Floor', 'Crop Floor',
                    'Remove Other Objects', 'Rescaling', 'Crop Ankle'
                ]
            )
            
            #################################################
            # Plot different orientation of final meshes
            #################################################
            # front, side, top, diagonal
            # directions = [np.array([0, 0]), np.array([0, 90]), np.array([90, 0]), np.array([45, 45])]
                
            # Define 9 diverse viewing directions
            # directions = [
            #     np.array([0, 0]), np.array([0, 90]), np.array([0, 180]), np.array([0, 270]),
            #     np.array([90, 0]), np.array([90, 90]), np.array([90, 180]), np.array([90, 270]),
            #     np.array([45, 45]), np.array([135, 135]), np.array([225, 225]), np.array([315, 315])
            # ]

            # 6 views
            directions = [None, np.array([0, 0]), np.array([0, 90]), np.array([0, 180]), np.array([90, 0]), np.array([45, 45])]

            plot_mesh_angles(
                subject, data_image, feet_length, file_name.replace('.ply', ''), volume,
                point1, point2, p_min, p_max,
                final_mesh,
                directions
            )
            
            plt.show()
            
            results = [subject, data_image, ply_path, file_name, column, feet_length, 
                       len(mesh.vertices),
                       len(reoriented_mesh.vertices),
                       len(cropped_mesh.vertices),
                       len(largest_object_mesh.vertices),
                       len(scaled_mesh.vertices),
                       len(cropped_ankle_mesh.vertices),
                       len(final_mesh.vertices),
                       volume,
                       ''
                       ]
            df_results.append(results)

        except Exception as error:
            print('ERROR')
            print(error)
            
            results = [subject, data_image, ply_path, file_name, column, feet_length, 
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       error
                       ]
            df_results.append(results)
            
columns = ['subject', 'data_image', 'ply_path', 'file_name', 'direction', 'feet_length',
           'points_mesh', 'points_reoriented', 'points_cropped', 'points_remove_large', 'points_scaled',
           'points_cropped_ankle', 'points_final', 'volume', 'error'
           ]
df_results = pd.DataFrame(df_results, columns=columns)
# df_results.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_3_final_volume.csv', index=False)
df_results.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_3_final_volume_perfect.csv', index=False)


############################################################
Subject 1 - 1D0L - Feet length 24.0cm
############################################################
scene_dense_mesh_refine.ply
[24.0] - 24.0

Load Mesh
Number of Vertices:  39520

Removing isolated vertices using DBSCAN clustering
Remaining vertices: 39520

Locating floor plane and reorientate to x-y plane
No of points: 39520

Cropping mesh below the bottom 10th percentile of Z values
No of points: 19950

Perform mesh clustering to remove small objects
Range: [ 7.81061873 10.8631419   1.98959883]
eps: 0.1
min_points 10
Counter:te neighbors.[=>                                      ] 2%
Counter({0: 13383, 1: 6495, -1: 45, 2: 27})
3 cluster found!
Precompute neighbors.[========================================] 100%
No of points: 13383===============================>] 97%

Scaling longest x-y axis based on foot length: 24.0cm
Scaling Factor: 5.6 - Estimated Distance: 4.3 - Target Length: 24.0
No of points: 13383

Cropping mesh f

RPly: Unable to open file


Remaining vertices: 12655

Locating floor plane and reorientate to x-y plane
No of points: 12655

Cropping mesh below the bottom 10th percentile of Z values
No of points: 10303

Perform mesh clustering to remove small objects
Range: [ 7.06030582 10.28057275  3.3143401 ]
eps: 0.1
min_points 10
Counter:
Counter({-1: 8360, 23: 151, 10: 106, 46: 82, 36: 73, 41: 72, 25: 58, 39: 46, 53: 44, 40: 44, 6: 43, 27: 42, 18: 39, 9: 38, 49: 32, 1: 30, 38: 30, 31: 27, 0: 26, 3: 26, 34: 26, 56: 26, 4: 25, 48: 24, 58: 24, 7: 23, 30: 23, 59: 23, 22: 22, 42: 22, 21: 20, 35: 20, 52: 19, 12: 18, 24: 17, 66: 17, 50: 17, 29: 16, 13: 16, 17: 16, 51: 16, 43: 16, 2: 15, 37: 15, 44: 15, 26: 15, 54: 15, 45: 14, 73: 14, 33: 13, 64: 13, 76: 13, 20: 12, 19: 11, 78: 11, 82: 11, 11: 10, 61: 10, 75: 10, 5: 10, 28: 10, 69: 10, 8: 10, 71: 10, 15: 10, 14: 10, 84: 10, 81: 10, 77: 10, 63: 10, 79: 10, 65: 10, 47: 10, 80: 10, 86: 10, 70: 10, 55: 10, 85: 10, 60: 10, 83: 9, 32: 9, 57: 9, 62: 8, 67: 8, 68: 8, 72: 8, 74: 7, 16: 5}